## Метод главных компонент [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

Метод главных компонент (PCA) - используется для снижения размерности данных за их счет проекции ортогональное подпространство признаков (возможен при наличии линейных зависимостей между признаками).

Предположим что размер пространства $n$, для того чтобы снизить размерность до $k$ (где $k < n$) необходимо расчитать дисперсию вдоль каждой оси и выбрать оси с минимальной потерей информации или дисперсии. Многомерный вариант дисперсии матрица ковариаций. Ковариация - мера зависимости двух случайных величин и по определению равна:
$$Cov(X_i, X_j) = E[(X_i - E(X_i))*(X_j - E(X_j))] = E(X_i X_j) - E(X_i)*E(X_j)$$
Если произветсти предварительную центровку матрицы исходного пространства (из каждого ряда/признака вычесть среднее значение), то удастся избавиться от второго компонента в формуле и свести ее к
$$Cov(X_i, X_j) = E(X_i X_j)$$
Стоит отметить что для $X_i = X_j$ формула примет вид
$$Cov(X_i, X_ш) = Var(X_i)$$

Таким образом матрица ковариаций центрированной матрицы представляет собой симметричную диагональную матрицу, где по диагонали распологаются дисперсии сообветствующих признаков, а в ячейках ковариация между соответствующими признаками.
$$Var(X) = E(X*X^T)$$

Максимальная вариация нашего набора данных будет достигаться вдоль собственного вектора этой матрицы, соответствующего максимальному собственному значению. Таким образом главные компоненты, на которые мы бы хотели спроецировать наши данные, являются просто собственными векторами соответствующих топ-$k$ штук собственных значений этой матрицы.

Задача практически решена, для уменьшения размерности осталось умножить нашу матрицу данных на эти компоненты и мы получим проекцию наших данных в ортогональном базисе этих компонент. Теперь если мы транспонируем полученную новую матрицу данных и матрицу векторов главных компонент и перемножим их, мы восстановим исходную выборку в том пространстве, из которого мы делали проекцию на компоненты. При этом если количество компонент было меньше размерности исходного пространства, мы потеряем часть информации при таком преобразовании.

### Реализация на python

Инициализация данных

In [235]:
import numpy as np
np.random.seed(123)
x1 = np.arange(1,11)
y = 2 * x + np.random.randn(10)*2
X = np.vstack((x,y))
X

array([[ 1.        ,  2.        ,  3.        ,  4.        ,  5.        ,
         6.        ,  7.        ,  8.        ,  9.        , 10.        ],
       [-0.17126121,  5.99469089,  6.565957  ,  4.98741057,  8.8427995 ,
        15.30287307,  9.14664151, 15.14217474, 20.53187252, 18.2665192 ]])

Центровка матрицы

In [219]:
m = X.mean(axis=1)
Xcentered = X.T - m
print(Xcentered)
print("Mean vector: ", m)

[[ -4.5        -10.63222899]
 [ -3.5         -4.46627689]
 [ -2.5         -3.89501078]
 [ -1.5         -5.47355721]
 [ -0.5         -1.61816828]
 [  0.5          4.84190529]
 [  1.5         -1.31432627]
 [  2.5          4.68120696]
 [  3.5         10.07090474]
 [  4.5          7.80555142]]
Mean vector:  [ 5.5        10.46096778]


Расчет матрицы ковариаций

In [236]:
covmat = np.dot(Xcentered.T, Xcentered)/10
covmat

array([[ 8.25      , 16.47595751],
       [16.47595751, 39.01762827]])

Расчет собственных значений и собственных векторов

In [275]:
L, W = np.linalg.eig(covmat)
L

array([ 1.09232099, 46.17530728])

Сортировка собственных векторов по собственным значениям

In [239]:
W_sort = np.array([w[1] for w in sorted(zip(L, W.T), key = lambda matrix: matrix[0], reverse=True)])
W_sort

array([[-0.39845545, -0.91718769],
       [-0.91718769,  0.39845545]])

Преобразование данных

In [271]:
Xnew = np.dot(W_sort[0], Xcentered.T)
Xnew

array([ 11.54479904,   5.49100824,   4.56859456,   5.61796245,
         1.68339175,  -4.64016365,   0.60780069,  -5.28968401,
       -10.6315039 ,  -8.95220518])

 Процент потеренной дисперсии можно определить по собственным значениям

In [276]:
L[1]/np.sum(L)

0.9768907171110274

Восстановление исходных данных с потерей информации

In [270]:
n = 9     #номер элемента случайной величины
Xrestored = np.dot(Xnew.reshape(10, 1), W_sort[0].reshape(1, 2)) + m
print('Restored: ', Xrestored.T)
print('Original: ', X)

Restored:  [[ 0.89991191  3.31207784  3.6796186   3.26149224  4.82924338  7.34889849
   5.2578185   7.60770342  9.73618067  9.06705494]
 [-0.12777976  5.42468262  6.2707091   5.30824179  8.91698159 14.71686875
   9.90350047 15.31260083 20.21205226 18.67182015]]
Original:  [[ 1.          2.          3.          4.          5.          6.
   7.          8.          9.         10.        ]
 [-0.17126121  5.99469089  6.565957    4.98741057  8.8427995  15.30287307
   9.14664151 15.14217474 20.53187252 18.2665192 ]]


### Проверка алгоритма в сравнении sklearn

In [225]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
rez_s = pca.fit_transform(X.T)
rez_s.T

array([[ 11.54479904,   5.49100824,   4.56859456,   5.61796245,
          1.68339175,  -4.64016365,   0.60780069,  -5.28968401,
        -10.6315039 ,  -8.95220518]])

In [272]:
pca.explained_variance_ratio_

array([0.97689072])

### ПЕРЕПИСАТЬ РЕАЛИЗАЦИЮ ПОД КЛАСС И ПОИГРАТЬ С ПОТЕРЕННОЙ ДИСПЕРСИЕЙ